In [16]:
import pandas as pd

df = pd.read_csv("../data/income/adult.data")


df.shape

df.info()


df.head()


# เนื่องจากข้อมูลก่อนหน้าไม่มี feature label จะทำการเรียกข้อมูลใหม่โดยใส่ feature label จาก adult.name
def read_adult_names_file(file_path):
    with open(file_path, "r") as file:
        content = file.read()

    feature_section = content.split(">50K, <=50K.")[1].strip()

    features = []

    for line in feature_section.split("\n"):
        if line.strip():
            parts = line.split(": ")
            if len(parts) == 2:
                feature_name = parts[0].strip()
                features.append(feature_name)

    features.append("income")

    return features


column_names = read_adult_names_file("../data/income/adult.names")
df = pd.read_csv(
    "../data/income/adult.data",
    header=None,
    names=column_names,
    sep=", ",
    engine="python",
)


df.head()
df.isnull().sum()


for column in df.columns:
    j = df[column].value_counts(dropna=False)
    print(column, ":", j)
    print("----------------------------------------")


for column in df.columns:
    df.drop(df[df[column] == "?"].index, inplace=True)


def map_data(x):
    convert = x.unique()
    return x.map(dict(zip(convert, range(1, len(convert) + 1))))


for column in df.columns:
    if df[column].dtype == "object":
        df[column] = map_data(df[column])
df.astype(int)

df_test = pd.read_csv(
    "../data/income/adult.test",
    header=None,
    names=column_names,
    sep=", ",
    engine="python",
)
df_test.isnull().sum()
df_test.dropna(inplace=True)

for column in df_test.columns:
    df_test.drop(df_test[df_test[column] == "?"].index, inplace=True)

for column in df_test.columns:
    if df_test[column].dtype == "object":
        df_test[column] = map_data(df_test[column])


print(df_test.info())

X_train = df.drop("income", axis=1)
y_train = df["income"]

X_test = df_test.drop("income", axis=1)
y_test = df_test["income"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   39              32560 non-null  int64 
 1    State-gov      32560 non-null  object
 2    77516          32560 non-null  int64 
 3    Bachelors      32560 non-null  object
 4    13             32560 non-null  int64 
 5    Never-married  32560 non-null  object
 6    Adm-clerical   32560 non-null  object
 7    Not-in-family  32560 non-null  object
 8    White          32560 non-null  object
 9    Male           32560 non-null  object
 10   2174           32560 non-null  int64 
 11   0              32560 non-null  int64 
 12   40             32560 non-null  int64 
 13   United-States  32560 non-null  object
 14   <=50K          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
age : age
36    898
31    888
34    886
23    877
35    876
     ... 
83      6
88      3
85 

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


if len(y_train.shape) > 1:
    y_train = y_train.flatten()
if len(y_test.shape) > 1:
    y_test = y_test.flatten()

model = RandomForestClassifier(
    n_estimators=100, random_state=50, class_weight="balanced"
)
model.fit(X_train_scaled, y_train)

rfc_result_trained = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, rfc_result_trained)
print(f"Test accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, rfc_result_trained))
joblib.dump(model, "../exported_models/random_forest/rf_income_model.pkl")
joblib.dump(scaler, "../exported_models/random_forest/_income_scaler.pkl")

Test accuracy: 0.8090

Classification Report:
              precision    recall  f1-score   support

           1       0.80      0.99      0.89     11360
           2       0.88      0.26      0.40      3700

    accuracy                           0.81     15060
   macro avg       0.84      0.62      0.64     15060
weighted avg       0.82      0.81      0.77     15060



['../exported_models/random_forest/_income_scaler.pkl']